TO DO: 
- add IDs to artworks based on artists df 
- keep only the year for date columns 

In [1]:
import csv
import pandas as pd
import pandas_profiling as pp

In [15]:
rhz_artists= pd.read_csv('./Rhizome_data/artists_complete.csv', dtype='string')
#set empty fields w/ missing
rhz_artists['artistLabel'] = rhz_artists['artistLabel'].fillna('')
rhz_artists['Nationality'] = rhz_artists['Nationality'].fillna('missing')
rhz_artists['Nationality Guessed'] = rhz_artists['Nationality Guessed'].fillna('missing')
rhz_artists['Gender'] = rhz_artists['Gender'].fillna('missing')
rhz_artists['Gender Guessed'] = rhz_artists['Gender Guessed'].fillna('missing')
rhz_artists['Birth'] = rhz_artists['Birth'].fillna('0').astype('int')
rhz_artists['Death'] = rhz_artists['Death'].fillna('0').astype('int')
rhz_artists['Wiki QID'] = rhz_artists['Wiki QID'].fillna('')
rhz_artists['ULAN'] = rhz_artists['ULAN'].fillna('')
#set collective fields to empty not missing 
rhz_artists['collectiveLabel'] = rhz_artists['collectiveLabel'].fillna('')
rhz_artists['collectivePage'] = rhz_artists['collectivePage'].fillna('')
#create and populate ID field to cross reference with artworks 
rhz_artists['ID'] = range(1, 1+len(rhz_artists))
#drop the notes column 
rhz_artists = rhz_artists.drop('Notes', axis=1)
#rename columns 
rhz_artists.rename(columns={'artistLabel': 'Artist', 'artistPage': 'Artist URL', 'collectiveLabel': 'Collective', 'collectivePage': 'Collective URL', 'Wiki QID': 'WikiQID'}, inplace=True)

In [17]:
rhz_artworks= pd.read_csv('./Rhizome_data/artwork_complete.csv', dtype='string')
#auto populate ID based on artist df 
#rhz_artworks['ID'] = TKTKTKTK 
#set empty fields w/ missing or empty strings 
rhz_artworks['accession'] = rhz_artworks['accession'].fillna('0')
rhz_artworks['inception'] = rhz_artworks['inception'].fillna('0')
url_columns = ['summary_url', 'summary_url_2', 'description_url', 'description_url_2', 'statement_url']
rhz_artworks[url_columns] = rhz_artworks[url_columns].fillna('')
#populate ID column
#rename columns 
rhz_artworks.rename(columns={'artwork_label': 'Title', 'artist_label': 'Artist', 'accession': 'DateAcquired', 'inception': 'DateCreated', 'artwork_page': 'URL'}, inplace=True)

In [ ]:
rhz_artists_ids = rhz_artists[['ID', 'Artist']]

            

Reports 

In [ ]:
pp.ProfileReport(rhz_artists)